In [1]:
#тут много лишний библиотек подключено, так как пробовали много идей и не все зашли
import pandas as pd;
import random
import numpy as np;
import matplotlib.pyplot as plt
import re
from tqdm import tqdm
import requests
from sklearn.metrics import accuracy_score
from sklearn.cross_validation import train_test_split
import scipy
from sklearn import metrics
from sklearn.metrics import log_loss
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
from sklearn.model_selection import cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.metrics import accuracy_score

from sklearn.model_selection import KFold
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
import warnings
warnings.simplefilter('ignore')
pd.set_option('display.max_columns', 30)
%matplotlib inline

#функция, которая по номеру определяет оператора и город
def get_info_tel(number):
    try:
        b = requests.get('http://www.megafon.ru/api/mfn/info/?msisdn='+str(number))
        data = b.json()
        if ('operator' in data) and 'region' in data:
            return {'operator':data['operator'],'region':data['region']}
        else:
            return -1
    except:
        return -1
#get_info_tel(89831699052)

#находит, указал ли клиент свой номер телефона в сообщении
def find_phone_number(text):
    num = re.findall(r'\+?[78][-\s]?\(?\d{3}\)?[-\s]?\d{3}[-\s]?\d{2}[-\s]?\d{2}', text)
    if num:
        num2 = re.findall(r'\d',num[0])
        if num2:
            return ''.join(num2)
        else: 
            return np.nan
    else:
        return np.nan

#считываем данные и удаляем данные с na
df = pd.read_csv('../data/email_reports.csv')
df = df.dropna(subset=['CONTENT'])

#две разные функции, который из емейлов достают тему обращения
def receive_problem_type_1(type_ms):
    return type_ms[type_ms.find('Выберите тему обращения: ')+ 25:].split('\r\n')[0]
def receive_problem_type_2(text):
    x = re.findall(r'Выберите тему обращения: \w+', text)
    if x != []:
        pr_type = x[0].split()[-1]
    else:
        pr_type = np.nan
    return  pr_type

#распарсим темы сообщения их письма
df['problem_type'] = df.CONTENT.apply(lambda x: receive_problem_type_1(x))
df_tmp = df.CONTENT.apply(lambda x: receive_problem_type_2(x))
# некоторые письма имеют нестандартный формат.
#было замечено, котогда функции выдают одинаковый ответ, то почта имеет стандартный формат
#этой строчкой удалим все письма плохого формата
df = df[(df['problem_type'] == df_tmp) & (df['problem_type'] != 'on') ]

#оставили только чистые данные, удалив темы, написанные с ошибками

#распарсим текст обращения
def receive_problem_text(text):
    try:
        x = re.split(r'Задайте ваш вопрос или опишите проблему: ', text)
        #print(x)
        if x != []:
            x2 = re.split(r'Выражаю согласие\* на обработку пе'+
                          r'рсональных данных в соответствии со ст\.9 ФЗ №152 "О персональных данных".: ',
                          x[1])
            if x2 != []:
                pr_text = x2[0]
            else:
                pr_text = x[1]
        else:
            pr_text = np.nan
    except IndexError:
            pr_text = np.nan
    return  pr_text

# удалим все данные с na
df['problem_text'] = df.CONTENT.apply(lambda x: receive_problem_text(x))
df = df.dropna(subset=['problem_text'])
df = df.dropna(subset=['SUBJECT'])

/usr/local/lib/python3.6/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
#узнаем, сколько в секундах работник отвечал на емейл
df['time'] = pd.to_datetime(df['treatment.end.time']) - pd.to_datetime(df['agent.receive.time'])
df['time_s'] = df['time'].apply(lambda x: x.seconds)
#удалим выбросы 
df = df[(df['time_s'] > 9) & (df['time_s'] < df['time_s'].quantile(0.95)) ]
df.head()

,SENDER_ID,SENDER_EMAIL,OPERATOR_ID,ACCEPTOR,SUBJECT,To.queue.time,agent.receive.time,treatment.end.time,CONTENT,ANSWERTEXT,problem_type,problem_text,time,time_s
0,token_7747,xxxx@yandex.ru,token_48,spb.vopros@tele2.ru,Обратная связь,2088-06-17 02:31:32,2088-06-18 19:48:26,2088-06-18 20:24:59,Тело письма\r\n\r\nMSISDN: +xxx xxx xx xx\r\nО...,"\r\n\r\nЗдравствуйте, xxxxx xxxxx xxxxx!\r\n\r...",прочее,"Здравствуйте. Я и моя жена xxxxx xxxxx xxxxx, ...",00:36:33,2193
1,token_14226,xxxx@mail.ru,token_50,Saransk.Vopros@tele2.ru,Re[2]: Обратная связь,2088-06-17 02:31:52,2088-06-18 18:43:45,2088-06-18 18:58:43,\r\nвся необходимая информация отображена в ...,"Здравствуйте, xxxxx xxxxx xxxxx!\r\nБлагодарим...",тарифыИУслуги,Добрый вечер.\r\nСотрудн...,00:14:58,898
2,token_23942,xxxx@mail.ru,token_3,rostov.vopros@tele2.ru,Обратная связь,2088-06-17 02:33:14,2088-06-18 18:43:45,2088-06-18 18:47:06,Тело письма\r\n\r\nMSISDN: +xxx xxx xx xx\r\nО...,"\r\n\r\nЗдравствуйте, xxxxx xxxxx xxxxx!\r\nБл...",мобильныйИнтернет,Интернет опять еле тянет. Преимущественно сеть...,00:03:21,201
3,token_18368,xxxx@mail.ru,token_53,krasnodar.vopros@tele2.ru,Обратная связь,2088-06-17 02:33:36,2088-06-18 19:49:10,2088-06-18 19:53:57,Тело письма\r\n\r\nMSISDN: +xxx xxx xx xx\r\nО...,"\r\n\r\nЗдравствуйте, xxxxx xxxxx xxxxx! ¦\r\n...",подключенныеУслуги,С номера 312 приходят странные смс уведомлении...,00:04:47,287
4,token_21039,xxxx@mail.ru,token_34,kostroma.vopros@tele2.ru,Обратная связь,2088-06-17 02:33:56,2088-06-18 19:49:17,2088-06-18 19:59:48,"Тело письма\r\n\r\nОтметьте, в каком регионе В...","\r\n\r\nАлександр, доброго времени суток! ;)\r...",мобильныйИнтернет,Здравствуйте. В каких регионах есть VoLTE? И е...,00:10:31,631


In [3]:
#составим список всех категорий
list_cat = list(df['problem_type'].unique())
#добавим для каждого столбца фичу, сколко по времени отвечал сотрудник
for i in list_cat:
    df[i] = (df['problem_type'] == i) * df['time_s']
    df.loc[df[i] == 0,i] = None

#сгруппируем данные по сотрудникам и узнаем, сколько в среднем по времени сотрудник отвечает на каждую категорию
df_operator_mean_cat = df.groupby('OPERATOR_ID').mean()[list_cat]
df_operator_mean_cat.head()

,прочее,тарифыИУслуги,мобильныйИнтернет,подключенныеУслуги,блокировкаИРазблокировкаSimКарты,роумингИЗонаОбслуживания,личныйКабинет,поступлениеПлатежейОшибочныеПлатежи,детализацияСчётаИлиЗвонков,подключениеКTele2,переносНомераВTele2,информацияОПодключенныхНомерах
OPERATOR_ID,,,,,,,,,,,,
token_1,495.168605,544.546584,455.976526,485.701493,482.636364,530.878505,384.469136,480.324074,553.559140,364.027778,476.903226,440.562500
token_10,515.166667,519.384615,499.444444,539.638298,497.727273,630.829268,363.230769,641.645161,467.560000,423.333333,433.700000,665.666667
token_100,479.214286,886.642857,499.818182,642.714286,NaN,510.714286,235.125000,340.000000,171.000000,351.000000,1198.000000,552.000000
token_101,433.295775,590.460784,370.604396,619.437500,335.000000,429.318182,483.508772,547.729730,318.702703,504.466667,454.727273,412.500000
token_102,660.371728,638.453488,614.971429,595.112500,561.294118,650.703704,604.018868,636.241379,543.660000,454.555556,667.944444,501.416667


In [4]:
# узнаем, сколько средний сотрудник отвечает по каждому типу сообщений
mean_cat = {}
for i in list_cat:
    df_tmp = df[df['problem_type'] == i]
    mean_cat[i] = df_tmp['time_s'].mean()
mean_cat

{'прочее': 559.9669485714286,
 'тарифыИУслуги': 600.9132807058277,
 'мобильныйИнтернет': 556.7802733053715,
 'подключенныеУслуги': 605.7724187209718,
 'блокировкаИРазблокировкаSimКарты': 538.5254314259764,
 'роумингИЗонаОбслуживания': 619.6975198573513,
 'личныйКабинет': 488.7150123528557,
 'поступлениеПлатежейОшибочныеПлатежи': 621.4545290182607,
 'детализацияСчётаИлиЗвонков': 533.8417845911949,
 'подключениеКTele2': 546.8960915157293,
 'переносНомераВTele2': 591.3749170537492,
 'информацияОПодключенныхНомерах': 494.1458333333333}

In [5]:
#узнаем для каждого сотрудника, на сколько в процентах он хуже/лучше среднего сотрудника
df_operator_mean_cat_proc = df_operator_mean_cat.copy()
for i in list_cat:
    df_operator_mean_cat_proc[i] = df_operator_mean_cat / mean_cat[i]
df_operator_mean_cat_proc.head()

,прочее,тарифыИУслуги,мобильныйИнтернет,подключенныеУслуги,блокировкаИРазблокировкаSimКарты,роумингИЗонаОбслуживания,личныйКабинет,поступлениеПлатежейОшибочныеПлатежи,детализацияСчётаИлиЗвонков,подключениеКTele2,переносНомераВTele2,информацияОПодключенныхНомерах
OPERATOR_ID,,,,,,,,,,,,
token_1,0.884282,0.824027,0.889343,0.817417,0.919490,0.799049,1.013205,0.796790,0.927557,0.905416,0.837318,1.002070
token_10,0.919995,0.857306,0.925260,0.850429,0.956625,0.831320,1.054125,0.828969,0.965018,0.941983,0.871134,1.042540
token_100,0.855790,0.797477,0.860688,0.791080,0.889864,0.773304,0.980560,0.771117,0.897671,0.876244,0.810339,0.969783
token_101,0.773788,0.721062,0.778217,0.715278,0.804597,0.699205,0.886602,0.697228,0.811656,0.792282,0.732692,0.876858
token_102,1.179305,1.098947,1.186054,1.090132,1.226259,1.065636,1.351241,1.062623,1.237018,1.207490,1.116672,1.336390


In [6]:
#так как мы будем предсказывать переменную problem_type, то создадим переменную y, в которой будут числа 0..11,
#которые обозначают каждую категорию обращения.
d = {}
for ii,i in enumerate(list_cat):
    d[i] = ii
df['y'] = df['problem_type'].map(d)   

#разобьем наши данные на трейн и тест
train,test = train_test_split(df,test_size = 0.2)
#сделаем tf idf
#также с помощью кроссвалидации было получено, что оптимальное кол-во фич = 10000
tff = TfidfVectorizer(ngram_range = (1,2),analyzer = 'word',max_features= 10000)
train_sparse = tff.fit_transform(train['problem_text'])
test_sparse = tff.transform(test['problem_text'])

y_train_time = train['time_s']
y_test_time = test['time_s']

y_train_cat = train['y']
y_test_cat = test['y']

In [7]:
%%time
#Чтобы предсказывать время ответа по входящему письму, обучим Ridge, с помощью GridSearchCV найдем опримальные параметры
grid ={'alpha':[1,10,100,1000,10000,100000]}
gr = GridSearchCV(Ridge(),grid,cv = 2,scoring='neg_mean_squared_error',n_jobs=6)
gr.fit(train_sparse,y_train_time)
for i in gr.grid_scores_:
    print(i)

mean: -275284.65124, std: 60.74864, params: {'alpha': 1}
mean: -255043.20817, std: 67.41905, params: {'alpha': 10}
mean: -255056.04751, std: 26.97139, params: {'alpha': 100}
mean: -259275.06399, std: 149.15836, params: {'alpha': 1000}
mean: -260741.66169, std: 155.39401, params: {'alpha': 10000}
mean: -260928.55173, std: 155.51968, params: {'alpha': 100000}
CPU times: user 2.34 s, sys: 192 ms, total: 2.53 s
Wall time: 7.66 s


In [8]:
# Оптимальный параметр alpha=100. посчитали ошибку на тесте
rg = Ridge(alpha=100)
rg.fit(train_sparse,y_train_time)
ans = rg.predict(test_sparse)
metrics.mean_squared_error(y_test_time,ans)

258677.70389238547

In [9]:
%%time
#чтобы по входящему письму определять тему обращения, обучим LogisticRegression на мультиклассовую классификацию  
grid ={'C':[0.25,0.5,1,2,5]}
gr = GridSearchCV(LogisticRegression(),grid,cv = 2,scoring='log_loss',n_jobs=-1)
gr.fit(train_sparse,y_train_cat)
for i in gr.grid_scores_:
    print(i)

mean: -1.33608, std: 0.00014, params: {'C': 0.25}
mean: -1.25937, std: 0.00046, params: {'C': 0.5}
mean: -1.21102, std: 0.00109, params: {'C': 1}
mean: -1.19043, std: 0.00148, params: {'C': 2}
CPU times: user 34.9 s, sys: 1.14 s, total: 36.1 s
Wall time: 57.5 s


In [10]:
#оптимальный параметр C=2, посмотрим ошибку предсказания на тесте, используя метрику log_loss.
lr = LogisticRegression(C=2)
lr.fit(train_sparse,y_train_cat)
ans = lr.predict_proba(test_sparse)
metrics.log_loss(y_test_cat,ans)

1.1458960053693366

In [11]:
#напишем телеграмм бота, который по входящему письму будет определять:
#1) тему письма
#2) прогнозируемое время ответа оператора
#3) есть ли номер клиента в письме
#4) если есть, то бот будет выводить регион клиента и к какому оператору он принадлежит

import time
import telebot
bot = telebot.TeleBot("676810517:AAHo3w8G_X-kq9nXwYar4voL9Pd1EUCAwbU")

d_cat = {
    0:'прочее',
    1:'тарифы и услуги',
    2:'мобильный интернет',
    3:'подключенные услуги',
    4:'блокировка и разблокировка sim карты',
    5:'роуминг и зона обслуживания',
    6:'личный кабинет',
    7:'поступление платежей, ошибочные платежи',
    8:'детализация счёта или звонков',
    9:'подключение к Tele2',
    10:'перенос номера в Tele2',
    11:'информация о подключенных номерах'
}

@bot.message_handler(commands=['start', 'help'])
def send_welcome(message):
	bot.reply_to(message, "Напишите свой вопрос")

In [100]:
@bot.message_handler(func=lambda message: True)
def echo_all(message):
	#bot.reply_to(message, message.text)
    #text = test.loc[1729,'problem_text']
    text = message.text
    text_sparse = tff.transform([text])
    cat = d_cat[lr.predict(text_sparse)[0]]
    time = int(rg.predict(text_sparse)[0])
    ans = 'Предпологаемая тема ображения: %s \n' % cat
    ans +='Предпологаемое время ответа сотрудника: %s \n' %time
    number = find_phone_number(text)
    if number is np.nan:
        ans +='Номера клиента в сообщении не обнаружено \n' 
    else:
        tel_info = get_info_tel(number)
        if tel_info != -1:
            ans +='Номера клиента: %s \n' % str(number)
            ans +='Оператор клиента: %s \n' % str(tel_info['operator'])
            ans +='Город клиента: %s \n' % str(tel_info['region'])
    bot.send_message(message.chat.id, ans)
bot.polling()


Не могу войти в приложение. Не приходит пароль,  пишет: -невозможно отправить sms 
Отправьте пожалуста пароль.
nan
Здравствуйте! Не могу отключить функцию «Мелодия для себя» ни из личного кабинета ни командой *130*202#. Что делать?
nan
Подключила услугу 1гб за 100р,подключили и сняли деньги,гигабайт пришёл, я решила отключить услугу т.к. она мне не нужна каждый месяц,отключила , и у меня исчез и тот гигабайт , который подключила сейчас!!!и деньги ушли!!!!прошу вернуть либо 1гб интернета , либо 100р,т.к. услуга не оказана
nan


In [12]:
#Промодулируем работу нашего алгоритма
#Пусть number_of_free_employees  - кол-во свободных сотрудников, измеряется от 0% до 100%, является случайной величиной с 1 - нормаль
#ное распределение
#Первый алгоритм будет получать тему письма, выбирать всех свободных сотрудников, и находить самого производительного сотрудника по 
#этой теме и давать ему данное обращение
#Второй алгоритм будет распределять тему обращения рандомно между сотрудниками. 
# Будем моделировать на 100000 сообщения, предпологам что вероятность появления каждой темы совпадает с историческими данными.

In [13]:
frequency_categories = df['problem_type'].value_counts(normalize = True)
frequency_categories

прочее                                 0.235565
тарифыИУслуги                          0.185523
мобильныйИнтернет                      0.159182
подключенныеУслуги                     0.090425
личныйКабинет                          0.074099
роумингИЗонаОбслуживания               0.066432
поступлениеПлатежейОшибочныеПлатежи    0.059561
детализацияСчётаИлиЗвонков             0.054791
блокировкаИРазблокировкаSimКарты       0.023713
подключениеКTele2                      0.022593
переносНомераВTele2                    0.016228
информацияОПодключенныхНомерах         0.011889
Name: problem_type, dtype: float64

In [120]:
time_1 = 0
time_2 = 0
for i in tqdm(range(100000)):
    number_of_free_employees = max(min(1.,1 - np.random.normal()),0)
    if number_of_free_employees != 1:
        data = train_test_split(df_operator_mean_cat_proc,test_size = number_of_free_employees)[0]
        if len(data) > 0:
            random_problem_type = np.random.choice(frequency_categories.index,1,list(frequency_categories))[0]
            time_1 += 1/np.max(data[random_problem_type])
            time_2 += 1/np.random.choice(data[random_problem_type])
        else:
            time_1 += 1
            time_2 += 1
    else:
        time_1 += 1
        time_2 += 1

100%|██████████| 100000/100000 [01:09<00:00, 1438.63it/s]


In [121]:
print('В первом случае 100к обращений обработано за %d единиц времени. Во втоором случае за %d единиц времени' %(time_1,time_2))
print('В первом случае сотрудники справились на %.2f быстрее'%(1 - time_1/float(time_2)))

В первом случае 100к обращений обработано за 70878 единиц времени. Во вотором случае за 103263 единиц вмерени
В первом случае сотрудники справились на 0.31 быстрее
